In [1]:
import numpy as np
import networkx as nx
import math
import scipy.sparse as sps
from heapq import *
from scipy.spatial.distance import euclidean
from sklearn import metrics
import matplotlib.pyplot as plt
import random
import pdb

from IndexedDistance import IndexedDistance

In [2]:
MONTHLY = 0
WEEKLY = 1

In [13]:
def combine_embeddings_sum(emb):
    '''
        Given embeddings emb, combine them by just adding them together.
    '''
    combined_embeddings = np.sum(emb, axis=0)

    return combined_embeddings

def combine_embeddings_exp_decay(emb, theta):
    '''
        Given embeddings emb, combine them with an exponential decay.
    '''
    decayed_embeddings = exp_decay(emb, theta)
    combined_embeddings = np.sum(decayed_embeddings, axis=0)

    return combined_embeddings

def exp_decay(emb, theta):
    '''
    Given a vector of size T (time),
    Multiple all entries with diminishing factors (where
    the latest entry will not be diminished)
    theta > 0
    '''
    T = len(emb)
    emb_dec = np.zeros_like(emb)
    for i in range(T):
        emb_dec[T-1 - i] = emb[T-1 - i] * math.pow(math.e, (-i * theta)) #/ 2.0

    return emb_dec

def perform_link_prediction(emb, test):
    '''
    emb: node embeddings
    test: Last snapshot graph whose edges are to be predicted.
    Returns prediction and ground truth as arrays
    '''
    # Get the num of edges in the test (T-th) graph. This number will be used
    # to generate our predicted edges.
    k = get_number_of_edges(test)

    # Get the ground truth connections
    truth = nx.to_numpy_matrix(test)

    pred_graph = get_prediction_graph(emb, k)
    # Get the corresponding numpy matrix, but make sure all node ids are included
    pred = nx.to_numpy_matrix(pred_graph, nodelist=list(range(0, len(emb))))

    # Now evaluate predictions wrt truth.
    truth_arr = np.asarray(truth.flatten()).reshape(-1)
    pred_arr = np.asarray(pred.flatten()).reshape(-1)
    
    return truth_arr, pred_arr
    # print(np.shape(truth_arr))
    # print(np.shape(pred_arr))

    # truth_arr = np.asarray(truth_arr)
    # pred_arr = pred_arr[0]

    
def evaluate(truth_arr, pred_arr):
    print(type(truth_arr))
    print(np.shape(truth_arr))
    print(np.shape(pred_arr))

    cm = metrics.confusion_matrix(truth_arr, pred_arr)
    print(cm)

    print(metrics.classification_report(truth_arr, pred_arr))
    
    map = metrics.average_precision_score(truth_arr, pred_arr)
    print('Mean Average Precision: {}'.format(map))
    
    fpr, tpr, thresholds = metrics.roc_curve(truth_arr, pred_arr)
    roc_auc = metrics.auc(fpr, tpr)
    print('Area Under ROC Curve: {}'.format(roc_auc))
    
    return mapscore, fpr, tpr, roc_auc

def get_number_of_edges(g):
    '''
    Assuming g is a networkx graph, return number of edges
    '''
    return g.number_of_edges()

def get_prediction_graph(emb, k):
    '''
    First, get k closest pairs from embedding matrix.
    Then, extract the predicted graph.
    '''
    print('Find k-closest edges as predictions...')
    k_closest = find_k_closest(emb, k)
    pred_graph = nx.Graph()
    print('Build the graph from the k-closest...')
    for i in range(len(k_closest)):
        v1, v2 = k_closest[i].get_indices()
        print('{},{}'.format(v1,v2))
        pred_graph.add_edge(v1, v2)

    return pred_graph

def find_k_closest(emb, k):
    '''
    Finds k closest pairs of the given embedding.
    Returns a list of custom data structure, IndexedDistance.
    '''
    # Create priority queue
    prio = []
    n = len(emb)

    # for i in range(k):
    #     v1 = random.randint(0, n-1)
    #     v2 = random.randint(0, n-1)
    #     d = 0.3
    #     id = IndexedDistance(v1, v2, d)
    #     prio.append(id)

    n = len(emb)
    for i in range(n):
        for j in range(i+1, n):
            d = euclidean(emb[i], emb[j])
            id = IndexedDistance(i, j, d)
            heappush(prio, id)

        if(len(prio) > 10*k):
            prio = prio[:10*k]

    # Now prio have the smallest k point pairs with their distances.
    return prio

def load_reality_mining_snapshots(mode : int = MONTHLY):
    '''
    Reality Mining Dataset, loaded from saved sparse matrices.
    '''
    if(mode == MONTHLY):
        T = 11      # For Reality Mining, there are 11 monthly snapshots
        path = './RM_sparse/Monthly/vc-month-'
        sparse_graphs = []
        for i in range(T):
            snp = sps.load_npz('{}{}.npz'.format(path, i))
            sparse_graphs.append(snp)

        train, test = sparse_graphs[:T-1], sparse_graphs[T-1]
        return train, test
    elif(mode == WEEKLY):
        T = 51      # For Reality Mining, there are 11 monthly snapshots
        path = './RM_sparse/Weekly/vc-week-'
        sparse_graphs = []
        for i in range(T):
            snp = sps.load_npz('{}{}.npz'.format(path, i))
            sparse_graphs.append(snp)

        train, test = sparse_graphs[:T-1], sparse_graphs[T-1]
        return train, test
    else:
        print('Invalid Processing Mode. Please use MONTHLY (0) or WEEKLY (1).')
        return None

def get_reality_mining_vc_embeddings(mode: int = 0):
    '''
    Load the pre-computed LINE embeddings
    '''
    if(mode == MONTHLY):
        T = 11
        emb_path = './RM_emb/vc_month/em-month-'
        embs = []
        for i in range(T-1):    # Read the embeddings, except the last one.
            emb = np.load('{}{}.npz'.format(emb_path, i))
            # print(list(emb.keys()))
            # print(emb['arr_0'])
            # Embeddings are in the arr0 key
            embs.append(emb['arr_0'])
            
    elif(mode == WEEKLY):
        T = 51
        emb_path = './RM_emb/vc_week/em-vc-week-'
        embs = []
        for i in range(T-1):    # Read the embeddings, except the last one.
            emb = np.load('{}{}.npy'.format(emb_path, i))
            # print(list(emb.keys()))
            # print(emb['arr_0'])
            # Embeddings are in the arr0 key
#             embs.append(emb['arr_0'])
            embs.append(emb)

    return np.array(embs)

In [14]:
# Load data. IF YOU HAVE THE EMBEDDINGS READY, YOU JUST NEED TO
# LOAD THE LAST SNAPSHOT'S DATA.
train, test = load_reality_mining_snapshots(mode=WEEKLY)

# Get the last snapshot graph as a networkx graph.
test_graph = nx.from_scipy_sparse_matrix(test)

# import embeddings
emb = get_reality_mining_vc_embeddings(mode=WEEKLY)

# Choose a combination scheme for embeddings...
# comb = combine_embeddings_sum(emb)
comb = combine_embeddings_exp_decay(emb, 0.9)
print(comb)

# Then, use combined embeddings for temporal link prediction task
# on the test graph, which is the T-th snapshot.
truth_arr, pred_arr = perform_link_prediction(comb, test_graph)


[[ 0.00091127  0.03289941  0.02977814 ...  0.01091341  0.01402635
  -0.02297924]
 [-0.00325191 -0.01002752 -0.02038736 ...  0.01906702 -0.04257497
   0.00576544]
 [-0.01712216 -0.01192139  0.05562051 ... -0.00578905  0.0287878
  -0.0152207 ]
 ...
 [ 0.02253347  0.03292423  0.01110934 ... -0.05307961  0.0404151
   0.00567906]
 [-0.03690622  0.02988864 -0.03454335 ... -0.05074416 -0.00240597
   0.02989971]
 [ 0.00923644 -0.01527099 -0.0359187  ...  0.02868086  0.03115165
   0.05351039]]
Find k-closest edges as predictions...
Build the graph from the k-closest...
3236,4450
3036,8646
1196,7215
1243,5326
280,6641
2511,8571
965,9009
5389,7297
3608,8421
1658,6046
4372,5723
3651,4310
3351,10010
1504,1841
1990,10010
2892,7261
642,9414
2611,3644
3547,5770
4467,9569
7095,7651
1842,4543
4031,9871
117,1606
253,6774
1065,7831
8275,8571
2103,9345
782,8646
5730,9472
1921,2795
3503,7465
3936,8082
239,9741
238,2302
471,2876
6477,6936
3336,5730
3014,4152
5481,6641
430,5500
707,1860
1292,2511
845,6041
471

In [15]:
mapscore, fpr, tpr, roc_auc = evaluate(truth_arr, pred_arr)
print(mapscore)

<class 'numpy.ndarray'>
(101143249,)
(101143249,)
[[101142941       280]
 [       28         0]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00 101143221
         1.0       0.00      0.00      0.00        28

    accuracy                           1.00 101143249
   macro avg       0.50      0.50      0.50 101143249
weighted avg       1.00      1.00      1.00 101143249

Mean Average Precision: 2.768350856516385e-07
Area Under ROC Curve: 0.49999861582418853


NameError: name 'mapscore' is not defined

In [ ]:

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


# COLLEGE MSG

In [ ]:
# Load data. IF YOU HAVE THE EMBEDDINGS READY, YOU JUST NEED TO
# LOAD THE LAST SNAPSHOT'S DATA.
test_graph = load_collegemsg_test_graph()

# at the end of this line, you should have a nx graph of the last snapshot.
# now yoi are ready to import embeddings for nodes.

# import embeddings BUT only for the first T-1 timestamps.
emb = get_collegemsg_embeddings()

# Choose a combination scheme for embeddings...
# comb = combine_embeddings_sum(emb)
combined_emb = combine_embeddings_exp_decay(emb, 0.9)
print(combined_emb)

# Then, use combined embeddings for temporal link prediction task
# on the test graph, which is the T-th snapshot.
truth_arr, pred_arr = perform_link_prediction(combined_emb, test_graph)


In [ ]:
mapscore, fpr, tpr, roc_auc = evaluate(truth_arr, pred_arr)
print(mapscore)